In [2]:
import pandas as pd
import numpy as np
import sqlite3
import os

PROCESSED_PATH = "C://Users//Rachit//OneDrive//Documents//CreditPathAI//data//processed"
DB_PATH = "C://Users//Rachit//OneDrive//Documents//CreditPathAI//data//processed//creditpath.db"


In [ ]:
# loading the data set from "Processed_data folder":
# loaded all cleaned files
files = [f for f in os.listdir(PROCESSED_PATH) if f.endswith("_clean.csv")]

data = {}

for f in files:
    path = os.path.join(PROCESSED_PATH, f)
    df = pd.read_csv(path)
    data[f.replace("_clean.csv", "")] = df

data.keys()


dict_keys(['HC_application_train', 'HC_bureau_balance', 'HC_bureau', 'HC_credit_card_balance', 'HC_installments_payments', 'HC_POS_CASH_balance', 'HC_previous_application', 'HC_sample_submission', 'HomeCredit_columns_description'])

In [4]:
# Extracted the main application table
app = data["HC_application_train"].copy()

# Renamed key column for safety
app.rename(columns={"sk_id_curr": "SK_ID_CURR"}, inplace=True)

app.shape


(307511, 122)

In [7]:
# Extracting tables
bureau = data["HC_bureau"].copy()
bureau_balance = data["HC_bureau_balance"].copy()

# Renaming key columns
bureau.rename(columns={"sk_id_curr": "SK_ID_CURR", "sk_id_bureau": "SK_ID_BUREAU"}, inplace=True)
bureau_balance.rename(columns={"sk_id_bureau": "SK_ID_BUREAU"}, inplace=True)

# --- Step 1: Aggregate bureau_balance at bureau level ---
bb_agg = bureau_balance.groupby("SK_ID_BUREAU").agg({
    "months_balance": "mean",
    "status": "nunique"
}).reset_index()

bb_agg.columns = ["SK_ID_BUREAU", "BB_MONTH_BAL_MEAN", "BB_STATUS_COUNT"]

# Merging this small aggregated table with bureau
bureau_merged = bureau.merge(bb_agg, on="SK_ID_BUREAU", how="left")

# --- Step 2: Aggregate at SK_ID_CURR (customer) level ---
bureau_final = bureau_merged.groupby("SK_ID_CURR").agg({
    "credit_active": "nunique",
    "credit_type": "nunique",
    "amt_credit_sum": "sum",
    "amt_credit_sum_debt": "sum",
    "amt_credit_sum_limit": "sum",
    "amt_credit_sum_overdue": "sum",
    "bb_month_bal_mean": "mean",
    "bb_status_count": "mean"
}).reset_index()

bureau_final.head()


KeyError: "Column(s) ['bb_month_bal_mean', 'bb_status_count'] do not exist"

In [6]:
bureau_balance = data["HC_bureau_balance"].copy()
print("Columns in bureau_balance:")
bureau_balance.columns


Columns in bureau_balance:


Index(['sk_id_bureau', 'months_balance', 'status'], dtype='object')